# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev1 toc-item"><a href="#Prepare-data" data-toc-modified-id="Prepare-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare data</a></div><div class="lev2 toc-item"><a href="#Install-Juicer" data-toc-modified-id="Install-Juicer-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Install Juicer</a></div><div class="lev2 toc-item"><a href="#Add-a-new-genome" data-toc-modified-id="Add-a-new-genome-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Add a new genome</a></div><div class="lev1 toc-item"><a href="#Run-Juicer-on-test1-sample" data-toc-modified-id="Run-Juicer-on-test1-sample-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Run Juicer on <code>test1</code> sample</a></div><div class="lev1 toc-item"><a href="#Run-Juicer-on-a-real-sample" data-toc-modified-id="Run-Juicer-on-a-real-sample-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run Juicer on a real sample</a></div>

# Introduction

Paths are relative to `projects/t47d/analysis/2017-05-26_process_hic_juicer`

# Prepare data

## Install Juicer

Adapt the instructions from the [Wiki's Installation page](https://github.com/theaidenlab/juicer/wiki/Installation):
```bash
# Clone repository
cd /software/mb/el7.2
git clone git@github.com:theaidenlab/juicer.git

# reference to the juicer scripts
ANALYSIS=$HOME/projects/t47d/analysis/2017-05-26_process_hic_juicer
JUICER=$ANALYSIS/data/juicer
mkdir -p $JUICER/scripts
cp -r /software/mb/el7.2/juicer/CPU/* $JUICER/scripts/

# miscellaneous scripts
cp /software/mb/el7.2/juicer/misc/* $JUICER/scripts/

# download Juicer tools jar (CUDA 7.0)
# source: https://github.com/theaidenlab/juicer/wiki/Download
# link to jar: http://hicfiles.tc4ga.com.s3.amazonaws.com/public/juicer/juicer_tools_0.7.0.jar
cd $JUICER/scripts/common
wget http://hicfiles.tc4ga.com.s3.amazonaws.com/public/juicer/juicer_tools_0.7.0.jar
chmod a+x juicer_tools_0.7.0.jar

# rename as I found that the script `juicer.sh` look for the second name
mv juicer_tools_0.7.0.jar juicer_tools.7.0.jar
```

**NOTE:** I tried to use the juicer scripts for UGER clusters, which includes the Univa cluster at the CRG. However, I run into problems relating to how software is internally loaded in the `juicer.sh` wrapper and in the CRG cluster.


## Add a new genome

Juicer runs on hg19 but it allows [adding a new genome](https://github.com/theaidenlab/juicer/wiki/Usage), so I will add our version of hg38 (which contains the mouse mammary virus tumoru, MMTV, as an additional scaffold). 

```
ANALYSIS=$HOME/projects/t47d/analysis/2017-05-26_process_hic_juicer

# directory to store reference FASTA and BWA-index files
REFERENCES=$ANALYSIS/data/juicer/references
mkdir -p $REFERENCES
IDIR=/users/GR/mb/jquilez/assemblies/homo_sapiens/hg38_mmtv/ucsc
ln -s $IDIR/hg38_mmtv_chr1-22XYMUn.* $REFERENCES/

# generate site positions
scripts/submit_generate_site_positions.sh
```
In the wrapper above I pass `hg38` instead of `hg38_mmtv` to the script `generate_site_positions.py` because from the code of [`generate_site_positions.py`](https://github.com/theaidenlab/juicer/blob/master/misc/generate_site_positions.py) I can see that only 4-characters tags are allowed.

# Run Juicer on `test1` sample

As a test, I will use the test FASTQs we use for the `hic-16.05` pipeline (2 x ~1,000 reads):
```
sample=test1
ANALYSIS=$HOME/projects/t47d/analysis/2017-05-26_process_hic_juicer
IDIR=/users/project/4DGenome/pipelines/hic-16.05/test
ODIR=$ANALYSIS/data/samples/${sample}/fastq
mkdir -p $ODIR
ln -s $IDIR/${sample}_read1.fastq.gz $ODIR/${sample}_R1.fastq.gz
ln -s $IDIR/${sample}_read2.fastq.gz $ODIR/${sample}_R2.fastq.gz
```

Adapt the usage instructions from the [Wiki's Usage page](https://github.com/theaidenlab/juicer/wiki/Usage):
```
sample=test1
ANALYSIS=$HOME/projects/t47d/analysis/2017-05-26_process_hic_juicer
DATA=$ANALYSIS/data
ifa=$DATA/juicer/references/hg38_mmtv_chr1-22XYMUn.fa
chrom_sizes=$DATA/juicer/references/hg38_mmtv_chr1-22XYMUn.chrom.sizes
restriction_enzyme=DpnII
assembly_version=hg38
SAMPLE=$DATA/samples/$sample
JUICER=$DATA/juicer
$JUICER/scripts/juicer.sh -z $ifa -p $chrom_sizes -s $restriction_enzyme -g hg38 -D $JUICER -d $SAMPLE

# delete intermediate files
cd $SAMPLE
bash ../../juicer/scripts/common/cleanup.sh
```

# Run Juicer on a real sample

The code above run fast because the `test1` sample had few reads. However, to process real samples with Juicer we need submit the job to the cluster.